In [ ]:
import pandas as pd
import numpy as np
import re
from typing import List


def get_crate_at(i: int, l: str):
    spaces = i
    left_braces = i + 1
    right_braces = i
    crate_ids = i
    j = spaces + left_braces + right_braces + crate_ids
    v = l[j : (j + 1)]
    return v if re.match(pattern=r"\w", string=v) else np.nan


def parse_initialstate_row(l: str, n_stacks=9):
    return [get_crate_at(i, l) for i in range(n_stacks)]


def parse_instruction_row(l: str, n_stacks=9):
    return [get_crate_at(i, l) for i in range(n_stacks)]


def read_stack_input(filename="input.txt", max_initial_crates=8):
    with open(filename) as f:
        a = f.readlines()
    return pd.DataFrame(
        [
            parse_initialstate_row(l.rstrip())
            for j, l in enumerate(a)
            if j < max_initial_crates
        ]
    )


df = read_stack_input()
N = df.shape[0]
heights = N - df.isnull().sum(axis=0)
df = df.iloc[::-1]
df
# Add n blank rows
n = 100
new_index = pd.RangeIndex(n)
empty = pd.DataFrame(None, index=new_index, columns=df.columns)
df = pd.concat([df, empty], axis=0).reset_index(drop=True)
df

df_init = df.copy()
heights_init = heights.copy()


def move(df: pd.DataFrame, heights: List[int], n: int, f: int, t: int):
    to_move = df.iloc[(heights[f] - n) : heights[f], f]
    df.iloc[heights[t] : (heights[t] + n), t] = to_move.iloc[::-1]
    df.iloc[(heights[f] - n) : heights[f], f] = np.nan
    heights[f] -= n
    heights[t] += n


def move2(df: pd.DataFrame, heights: List[int], n: int, f: int, t: int):
    to_move = df.iloc[(heights[f] - n) : heights[f], f]
    df.iloc[heights[t] : (heights[t] + n), t] = to_move
    df.iloc[(heights[f] - n) : heights[f], f] = np.nan
    heights[f] -= n
    heights[t] += n


def read_instruction_input(filename="input.txt", max_initial_crates=8):
    with open(filename) as f:
        a = f.readlines()
    return [
        list(map(int, re.findall(r"\d+", l)))
        for j, l in enumerate(a)
        if j > max_initial_crates + 1
    ]


inst = read_instruction_input()

# PART 1
for n, f, t in inst:
    move(df, heights, n, f - 1, t - 1)

ii = np.array(heights) - 1
print("".join([df.iloc[i, j] for j, i in enumerate(ii)]))

# PART 2
df = df_init.copy()
heights = heights_init.copy()
for n, f, t in inst:
  
    move2(df, heights, n, f - 1, t - 1)

ii = np.array(heights) - 1
print("".join([df.iloc[i, j] for j, i in enumerate(ii)]))
